## Select the analysis mode from the dropdown menu below.

In [1]:
import ipywidgets as widgets
from ipywidgets import interact
from measurement_directory import *

def set_analysis_type(analysis_mode):
    global analysis_type
    analysis_type = analysis_mode
    print('analysis mode is {a_type} \n'.format(a_type = analysis_type))

mode_select_widget = interact(set_analysis_type,analysis_mode=['fake analysis' , 'fake analysis 2', 'ycam', 'zcam_dual_imaging'])

interactive(children=(Dropdown(description='analysis_mode', options=('fake analysis', 'fake analysis 2', 'ycam…

## Enter the run from today you wish to analyze.

In [9]:
global watchfile
measurement_dir = measurement_directory()
watchfile = measurement_dir + r'\image_log.csv'

Enter name for this set of runs:run1b_after_cicero_crash


## Run the cell below to begin analyzing.

Check the output; it will update as images are analyzed.

To stop the analysis, press the interrupt kernel/stop button at the very top. You can restart the analysis by simply running the cell below again. The console output will be cleared, but not to worry, it's all in the log file.

In [3]:
print('loading matlab engine...')
import matlab.engine
eng = matlab.engine.start_matlab()
print('matlab engine loaded')

loading matlab engine...
matlab engine loaded


In [11]:
import os
import time
import datetime
import shutil
import posixpath
import sys
# from breadboard import BreadboardClient
import warnings
import pandas as pd
from numpy import isnan
import ipywidgets as widgets
from ipywidgets import interact


# analysis_type = 'fake analysis'

# define analysis_function which accepts image filepath and returns dictionary of {analyzed_var_name:analysis_value}
if analysis_type == 'fake analysis':
    from random import randint
    def analysis_function(filepath):
        time.sleep(randint(0,5))
        return {'fake analysis 1': randint(0,42), 'fake analysis 2': randint(0,42)}
    analyzed_var_names = ['fake analysis 1', 'fake analysis 2']
elif analysis_type == 'fake analysis 2':
    from random import randint
    def analysis_function(filepath):
        time.sleep(randint(0,5))
        return {'fake analysis 3': randint(0,42), 'fake analysis 4': randint(0,42)}
    analyzed_var_names = ['fake analysis 3', 'fake analysis 4']
elif analysis_type == 'ycam':
    analyzed_var_names = ['bareNcntAverageMarqueeBoxValues', 
                          'COMX', 'COMY'] #TODO set MATLAB analyzed_var_names properly
    from matlab_wrapper import getMATLABanalysis
    def analysis_function(filepath, previous_settings = None):
        if previous_settings is None:
            matlab_dict = getMATLABanalysis(eng, filepath)
        else:
#             print('previous settings: ' + previous_settings)
#             return {'fakematlab1': randint(0,42), 'fakematlab2': randint(0,42)}, 'foo1'
            matlab_dict = getMATLABanalysis(eng, filepath, marqueeBox = previous_settings['marqueeBox'],
                                           normBox = previous_settings['normBox'])
        analysis_dict, settings = matlab_dict['analysis'], matlab_dict['settings']
        return analysis_dict, settings
elif analysis_type == 'zcam_dual_imaging':
    analyzed_var_names = ['K_bareNcntAverageMarqueeBoxValues', 'Na_bareNcntAverageMarqueeBoxValues',
                          'Na_COMX', 'Na_COMY'] #TODO set MATLAB analyzed_var_names properly
    from dual_imaging_wrapper import getDualImagingAnalysis
    def analysis_function(filepath, previous_settings = None):
        if previous_settings is None:
            matlab_dict = getDualImagingAnalysis(eng, filepath)
#         else:
#             matlab_dict = getMATLABanalysis(eng, filepath, marqueeBox = previous_settings['marqueeBox'],
#                                            normBox = previous_settings['normBox'])
        analysis_dict = matlab_dict['analysis']
        settings = None
        return analysis_dict, settings 

warnings.filterwarnings(
    "ignore", "Your application has authenticated using end user credentials")
warnings.filterwarnings(
    "ignore", "Could not find appropriate MS Visual C Runtime")
# bc = BreadboardClient(config_path='API_CONFIG_fermi1.json')

def translate_stringy_list(stringy_list):
    #pd.read_csv imports listboundvariables dtype as a string, we want it in list form
    final_list = []
    stringy_list = stringy_list.split(',')
    for var in stringy_list:
        final_list.append(var.translate({ord(c): None for c in '[ ]\,\''}))
    return final_list

def analysis_to_imagelog(image_log_df, idx, outputlog_filename, previous_settings = None,
                        output_previous_settings = True):
    image_filename = image_log_df.loc[idx, 'filename0']
    abs_image_path = os.path.join(os.path.join(os.getcwd(), measurement_dir), image_filename)
    print(abs_image_path)
    settings = None
    if analysis_type == 'fake analysis' or analysis_type == 'fake analysis 2':
        analysis_dict = analysis_function(image_filename)
    elif (analysis_type == 'ycam' or analysis_type == 'zcam_dual_imaging') and previous_settings is None:
        analysis_dict, settings = analysis_function(abs_image_path)#no marquee box yet
    elif (analysis_type == 'ycam' or analysis_type == 'zcam_dual_imaging'):
        analysis_dict, settings = analysis_function(abs_image_path, previous_settings)
    if not output_previous_settings:
        settings = None #forces user to select new marquee box for each shot
    image_log_df = pd.read_csv(outputlog_filename) #get updated copy of log to prevent overwriting
    print('{file} analyzing: '.format(file=image_filename))
    for key in analyzed_var_names:
        print(key, analysis_dict[key])
        image_log_df.loc[idx,key] = analysis_dict[key]
    image_log_df.to_csv(outputlog_filename, index = False)
#     print('{file} analyzed: '.format(file=image_filename) + str(analysis_dict))
    return image_log_df, settings

def main():
    # Global settings
    # bc = BreadboardClient(config_path='API_CONFIG.json') # enter your path to the API_config
    refresh_time = 1  # seconds
    global watchfile
    previous_settings = None
#     watchfile = r'07\200729\foo\image_log.csv'
    print("\n\n Watching this file for changes: " + watchfile + "\n\n")

#     length_old = 0

    # Main Loop
    while True:
        if not os.path.exists(watchfile):
            time.sleep(refresh_time)
            continue
        else:
            image_log_df = pd.read_csv(watchfile)

        for var_name in analyzed_var_names:
            if var_name not in image_log_df.columns: #start new analysis type
#             print('{analysis} analysis started'.format(analysis = analysis_type))
                image_log_df, previous_settings = analysis_to_imagelog(image_log_df, 0, watchfile, previous_settings)
                if 'analyzed_variables' not in image_log_df.columns:
                    image_log_df.loc[0,'analyzed_variables'] = str(analyzed_var_names)
                    image_log_df.to_csv(watchfile, index = False)
                else:
                    old_analyzed_variables = set(translate_stringy_list(image_log_df.loc[0,'analyzed_variables']))
                    updated_analyzed_variables = list(old_analyzed_variables.union(set(analyzed_var_names)))
                    image_log_df.loc[0,'analyzed_variables'] = str(updated_analyzed_variables)
                    image_log_df.to_csv(watchfile, index = False)
            else: #queue un-analyzed images
                for idx in reversed(range(len(image_log_df.index))): #start from most recent image first
                    if isnan(image_log_df.loc[idx, var_name]):
                        image_log_df, previous_settings = analysis_to_imagelog(image_log_df, idx, watchfile, previous_settings)
                        if isinstance(image_log_df.loc[idx,'analyzed_variables'],float):
                            image_log_df.loc[idx,'analyzed_variables'] = str(analyzed_var_names)
                            image_log_df.to_csv(watchfile, index = False)
                        else:
                            old_analyzed_variables = set(translate_stringy_list(image_log_df.loc[idx,'analyzed_variables']))
                            updated_analyzed_variables = list(old_analyzed_variables.union(set(analyzed_var_names)))
                            image_log_df.loc[idx,'analyzed_variables'] = str(updated_analyzed_variables)
                            image_log_df.to_csv(watchfile, index = False)

            # # Write to Breadboard
            # try:
            #     resp = bc.post_images(
            #                 image_names = os.path.splitext(output_filename)[0],
            #                 filepath = output_filepath,
            #                 image_times = [dt],
            #                 auto_time = False # Add more information here
            #                 )
            #     if resp.status_code!=200:
            #         print(resp.text)
            # except: pass
            # names_old = names
#             length_old = len(image_log_df.index)

        # Wait before checking again
        time.sleep(refresh_time)

main()



 Watching this file for changes: 08\200806\run1b_after_cicero_crash\image_log.csv


D:\Fermidata1\enrico\08\200806\run1b_after_cicero_crash\dual run_1_run1_PA_m1500_m500_0_1337.spe
dual run_1_run1_PA_m1500_m500_0_1337.spe analyzing: 
K_bareNcntAverageMarqueeBoxValues 208.19969741132658
Na_bareNcntAverageMarqueeBoxValues 562.1633743803884
Na_COMX 115.38393518121138
Na_COMY 41.893784644982155
D:\Fermidata1\enrico\08\200806\run1b_after_cicero_crash\dual run_1_run1_PA_m1500_m500_0_1336.spe
dual run_1_run1_PA_m1500_m500_0_1336.spe analyzing: 
K_bareNcntAverageMarqueeBoxValues 54.28968021324355
Na_bareNcntAverageMarqueeBoxValues 480.70589478536647
Na_COMX 112.57456801164562
Na_COMY 46.161222985905695
D:\Fermidata1\enrico\08\200806\run1b_after_cicero_crash\dual run_1_run1_PA_m1500_m500_0_1338.spe
dual run_1_run1_PA_m1500_m500_0_1338.spe analyzing: 
K_bareNcntAverageMarqueeBoxValues 112.3864262458631
Na_bareNcntAverageMarqueeBoxValues 552.3184216739152
Na_COMX 114.57704593371386
Na_COMY 45.5

dual run_1_run1_PA_m1500_m500_0_1362.spe analyzing: 
K_bareNcntAverageMarqueeBoxValues -25.34654569804442
Na_bareNcntAverageMarqueeBoxValues 507.9972363133865
Na_COMX 111.47489730161426
Na_COMY 48.02132102530358
D:\Fermidata1\enrico\08\200806\run1b_after_cicero_crash\dual run_1_run1_PA_m1500_m500_0_1363.spe
dual run_1_run1_PA_m1500_m500_0_1363.spe analyzing: 
K_bareNcntAverageMarqueeBoxValues 209.8469992987424
Na_bareNcntAverageMarqueeBoxValues 624.6863696137641
Na_COMX 112.2542815417588
Na_COMY 43.78270108082131
D:\Fermidata1\enrico\08\200806\run1b_after_cicero_crash\dual run_1_run1_PA_m1500_m500_0_1364.spe
dual run_1_run1_PA_m1500_m500_0_1364.spe analyzing: 
K_bareNcntAverageMarqueeBoxValues 52.18575783113676
Na_bareNcntAverageMarqueeBoxValues 666.4388109972091
Na_COMX 112.66896789662108
Na_COMY 45.35461115630915
D:\Fermidata1\enrico\08\200806\run1b_after_cicero_crash\dual run_1_run1_PA_m1500_m500_0_1365.spe
dual run_1_run1_PA_m1500_m500_0_1365.spe analyzing: 
K_bareNcntAverageMarque

D:\Fermidata1\enrico\08\200806\run1b_after_cicero_crash\dual run_1_run1_PA_m1500_m500_0_1389.spe
dual run_1_run1_PA_m1500_m500_0_1389.spe analyzing: 
K_bareNcntAverageMarqueeBoxValues 219.51791285666542
Na_bareNcntAverageMarqueeBoxValues 622.950489380434
Na_COMX 112.51106290515101
Na_COMY 48.03814610800771
D:\Fermidata1\enrico\08\200806\run1b_after_cicero_crash\dual run_1_run1_PA_m1500_m500_0_1390.spe
dual run_1_run1_PA_m1500_m500_0_1390.spe analyzing: 
K_bareNcntAverageMarqueeBoxValues 178.5319135075917
Na_bareNcntAverageMarqueeBoxValues 564.5213043244925
Na_COMX 112.32277580088974
Na_COMY 49.83984768325658
D:\Fermidata1\enrico\08\200806\run1b_after_cicero_crash\dual run_1_run1_PA_m1500_m500_0_1391.spe
dual run_1_run1_PA_m1500_m500_0_1391.spe analyzing: 
K_bareNcntAverageMarqueeBoxValues 221.28386836278437
Na_bareNcntAverageMarqueeBoxValues 553.0408922690597
Na_COMX 114.41539202641262
Na_COMY 46.099121625550836
D:\Fermidata1\enrico\08\200806\run1b_after_cicero_crash\dual run_1_run1_PA

EmptyDataError: No columns to parse from file